In [1]:
#Let us first import necessary libararies
import numpy as np
import glob
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Data Conversion

The lidar data from the KITTI dataset is in `.bin` format which is currently not an accepted format in Open3D, hence this is converted to its `.pcd` equivalent. This has already been done for you and is located in the DATA folder of the main repo. However, should you come to work with the KITTI dataset yourself, here is the code to convert `.bin` files to `.pcd`, courtesy of [this Gist](https://gist.github.com/HTLife/e8f1c4ff1737710e34258ef965b48344).

However, this version has some errors, and thus, I have modified it for your convenience:

```python
def convert_kitti_bin_to_pcd(binFilePath, output_name):
    size_float = 4
    list_pcd = []
    with open(binFilePath, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float * 4)
    np_pcd = np.asarray(list_pcd)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_pcd)
    o3d.io.write_point_cloud(output_name, pcd)

# Data Visualization

In this first part we will observe the 3D pointcloud data. Per the kitty dataset its format is x, y, z, and intensity.  

In [2]:
#Lets first load the data
lidar_data_path = "../DATA/lidar_data/*.pcd"
lidar_file_names = sorted(glob.glob(lidar_data_path))
print(lidar_file_names[0:3])

['../DATA/lidar_data\\0000000000.pcd', '../DATA/lidar_data\\0000000001.pcd', '../DATA/lidar_data\\0000000002.pcd']


In [3]:
#We only need to work with one let us use the first one
lidar_file = lidar_file_names[0]
print(lidar_file)

../DATA/lidar_data\0000000000.pcd


In [5]:
#Read lidar file and create pointcloud data for vizualization
pcd = o3d.io.read_point_cloud(lidar_file)

Note: To visualized in open3d there is no need do as below, however this is was done to make the visualizer more asthetically pleasing. To simply visuzlize your point cloud all you need to do is

```python
# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [7]:
# Create a visualizer object
viz = o3d.visualization.Visualizer()
viz.create_window()

# Add the point cloud to the viz object
viz.add_geometry(pcd)

#Set the background color to black
opt = viz.get_render_option()
opt.background_color = np.array([0, 0, 0])  # RGB values for black

#Run the visualizer
viz.run()
viz.destroy_window()

### Output

![Point Cloud Visualization](../DOC_IMAGES/UNSUPERVISED_DOC_IMAGES/output_1.png)


# Plane segmentation

# DBSCAN clustering

# Voxel downsampling